## demo for parsing

In [2]:
%load_ext autoreload
%autoreload 2
from tree_sitter_languages import get_parser
parser = get_parser("java")

with open("grammar_test.java", "r") as file:
    source_code = file.read()
print(source_code)

import utils
tree = parser.parse(bytes(source_code, "utf8"))
print(utils.format_code(tree.root_node.sexp()))


//4sum
class Solution {
    String s = "hello".substring(0, 2);
    char c = 'c';
    int a = 0xAAAAAAAA;
    long l = 1L;
}
(program
    (line_comment)
    (class_declaration
        name: (identifier)
        body: (class_body
            (field_declaration
                type: (type_identifier)
                declarator: (variable_declarator
                    name: (identifier)
                    value: (method_invocation
                        object: (string_literal
                            (string_fragment)
                        )
                        name: (identifier)
                        arguments: (argument_list
                            (decimal_integer_literal)
                            (decimal_integer_literal)
                        )
                    )
                )
            )
            (field_declaration
                type: (integral_type)
                declarator: (variable_declarator
                    name: (identifier)
       

## Parse all solutions

In [3]:
# get program model from a java file
def getProgram(file_path):
    with open(file_path, "r") as file:
        source_code = file.read()
        utils.source_code_line = source_code.split("\n")
    tree = parser.parse(bytes(source_code, "utf8"))
    return utils.visit_program(tree.root_node)

# print program as a java file
def printJava(program):
    return utils.addIndentation(program.toString())

# validate the parser and the java generation
def validateJava(file_path):
    return_value = True
    try:
        program = getProgram(file_path)
        javaCode= printJava(program)
        tree = parser.parse(bytes(javaCode, "utf8"))
    except Exception as e:
        print(f"\nError in parsing java file: {file_path}")
        #print(e)
        return_value = False
    finally:
        return return_value
    
test_filepath = "solutions/01-matrix_1.java"
a = getProgram(test_filepath)
print(a.toString())
validateJava(test_filepath)

class Solution {
public int[][] updateMatrix(int[][] mat) {
int m = mat.length;
int n = mat[0].length;
int[][] dp = new int[m][n];
for ( int row = 0;row < m; row++ ) {
for ( int col = 0;col < n; col++ ) {
dp[row][col] = mat[row][col];
}
}
for ( int row = 0;row < m; row++ ) {
for ( int col = 0;col < n; col++ ) {
if ( dp[row][col] == 0 ) {
continue;
}
int minNeighbor = m * n;
if ( row > 0 ) {
minNeighbor = Math.min(minNeighbor, dp[row - 1][col]);
}
if ( col > 0 ) {
minNeighbor = Math.min(minNeighbor, dp[row][col - 1]);
}
dp[row][col] = minNeighbor + 1;
}
}
for ( int row = m - 1;row >= 0; row-- ) {
for ( int col = n - 1;col >= 0; col-- ) {
if ( dp[row][col] == 0 ) {
continue;
}
int minNeighbor = m * n;
if ( row < m - 1 ) {
minNeighbor = Math.min(minNeighbor, dp[row + 1][col]);
}
if ( col < n - 1 ) {
minNeighbor = Math.min(minNeighbor, dp[row][col + 1]);
}
dp[row][col] = Math.min(dp[row][col], minNeighbor + 1);
}
}
return dp;
}
}


True

In [4]:
import os
import subprocess
# Path to the solutions folder
folder_path = "mbjp"

# Get the list of Java code files in the folder
java_files = [file for file in os.listdir(folder_path) if file.endswith(".java")]

java_files.sort()
error_cnt=0
# Loop through each Java code file
for file_name in java_files[:1000]:
    file_path = os.path.join(folder_path, file_name)
    
    # using the parser to validate the java file
    if not validateJava(file_path):
        error_cnt+=1
print(f"Total errors: {error_cnt} out of {len(java_files)} files.")


Error in parsing java file: mbjp/MBJP_115.java

Error in parsing java file: mbjp/MBJP_119.java

Error in parsing java file: mbjp/MBJP_133.java

Error in parsing java file: mbjp/MBJP_136.java

Error in parsing java file: mbjp/MBJP_143.java

Error in parsing java file: mbjp/MBJP_158.java

Error in parsing java file: mbjp/MBJP_215.java

Error in parsing java file: mbjp/MBJP_234.java

Error in parsing java file: mbjp/MBJP_253.java

Error in parsing java file: mbjp/MBJP_269.java

Error in parsing java file: mbjp/MBJP_277.java

Error in parsing java file: mbjp/MBJP_278.java

Error in parsing java file: mbjp/MBJP_294.java

Error in parsing java file: mbjp/MBJP_297.java

Error in parsing java file: mbjp/MBJP_326.java

Error in parsing java file: mbjp/MBJP_333.java

Error in parsing java file: mbjp/MBJP_361.java

Error in parsing java file: mbjp/MBJP_37.java

Error in parsing java file: mbjp/MBJP_370.java

Error in parsing java file: mbjp/MBJP_399.java

Error in parsing java file: mbjp/MBJP_41

## Translating Program Model into Coq Proof

In [6]:
import shutil
import os
# coq proof test
def validateCoqProof(file_path):
    program=getProgram(file_path)
    coqProof=program.toCoq().toString()
    prefix="""From PLF Require Import Syntax.
Open Scope string_scope.

Example prog_well_typed : exists p, program_well_typed p.
Proof with (simpl;try(reflexivity)).
unfold program_well_typed.
eexists.
eexists."""
    suffix="""  Unshelve.
  all: apply STyVoid.
Defined.
Definition prog := the_exists_term (prog_well_typed).
Print prog."""
    coqProof=f"{prefix}\n{coqProof}\n{suffix}"
    with open("../coq_code/test.v", "w") as file:
        file.write(coqProof)
    res = subprocess.run(
                # coqc -Q ../coq_code PLF ../coq_code/test.v
                ["coqc", "-Q","../coq_code", "PLF", "../coq_code/test.v"],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=60,
            )
    if res.returncode != 0:
        def copy_and_rename_test_v(file_path):
            # Get the directory path and file name
            directory = os.path.dirname(file_path)
            file_name = os.path.basename(file_path)
            
            # Create the new file name with the same prefix as the original file
            new_file_name = file_name.split(".")[0] + ".v"
            
            # Create the new file path
            new_file_path = os.path.join("../coq_code", new_file_name)
            
            # Copy the test.v file to the new file path
            shutil.copyfile("../coq_code/test.v", new_file_path)
            shutil.copyfile(file_path, os.path.join("../coq_code", file_name))
            
            return new_file_path

        copy_and_rename_test_v(file_path)
        
        print(f"\nError in parsing coq proof: {file_path}")
        print(res.stderr.decode("utf-8"))
        return False
    else:
        return True

validateOne=0
if validateOne:
    validateCoqProof("mbjp/MBJP_219.java")
else:   
    error_cnt=0
    # Loop through each Java code file
    for file_name in java_files[:]:
        file_path = os.path.join(folder_path, file_name)
        print(file_name)
        # using the parser to validate the java file
        if not validateJava(file_path):
            continue

        # using coqc to validate the coq proof
        if not validateCoqProof(file_path):
            error_cnt+=1
    print(f"Total errors: {error_cnt} out of {len(java_files)} files.")

MBJP_1.java
MBJP_10.java
MBJP_100.java
MBJP_101.java
MBJP_102.java
MBJP_103.java
MBJP_104.java
MBJP_105.java
MBJP_106.java
MBJP_107.java
MBJP_108.java
MBJP_109.java
MBJP_11.java
MBJP_110.java
MBJP_111.java
MBJP_112.java
MBJP_113.java
MBJP_115.java

Error in parsing java file: mbjp/MBJP_115.java
MBJP_116.java
MBJP_117.java
MBJP_118.java
MBJP_119.java

Error in parsing java file: mbjp/MBJP_119.java
MBJP_12.java
MBJP_120.java
MBJP_121.java
MBJP_122.java
MBJP_123.java
MBJP_125.java
MBJP_126.java
MBJP_127.java
MBJP_128.java
MBJP_129.java
MBJP_130.java
MBJP_131.java
MBJP_132.java
MBJP_133.java

Error in parsing java file: mbjp/MBJP_133.java
MBJP_134.java
MBJP_135.java
MBJP_136.java

Error in parsing java file: mbjp/MBJP_136.java
MBJP_137.java
MBJP_138.java
MBJP_139.java
MBJP_14.java
MBJP_140.java
MBJP_141.java
MBJP_142.java
MBJP_143.java

Error in parsing java file: mbjp/MBJP_143.java
MBJP_144.java
MBJP_145.java
MBJP_146.java
MBJP_147.java
MBJP_148.java
MBJP_149.java
MBJP_15.java
MBJP_150.ja